# Create n-gram LM from Dubliners

> "For a student project"

- toc: false
- branch: master
- badges: false
- comments: true
- categories: [lm, dubliners, dt2112]

Version on [Kaggle](https://www.kaggle.com/code/jimregan/dubliners-lm)

In [ ]:
%pip install mosestokenizer

In [2]:
!pwd

/kaggle/working


In [3]:
HTML = "https://www.gutenberg.org/cache/epub/2814/pg2814-images.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
dubliners = requests.get(HTML)

In [6]:
assert dubliners.status_code == 200

In [7]:
soup = BeautifulSoup(dubliners.text, 'html.parser')

In [8]:
body = soup.find("body")

In [9]:
text = []
for chapter in body.findAll("div", {"class": "chapter"}):
    for element in chapter.findChildren():
        if element.name == "section" and element.get("id") in ["pg-header", "pg-footer"]:
            continue
        stripped = element.text.strip()
        if stripped != "":
            text.append(stripped)

In [10]:
from mosestokenizer import MosesSentenceSplitter

In [11]:
sents = []
try:
    with MosesSentenceSplitter('en') as splitsents:
        for para in text:
          if para == "":
              continue
          sents += splitsents([para.replace("\r\n", " ")])
except Exception as ex:
    print(ex, para)

In [12]:
import re
def cleaner(text):
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("’ ", " ")
    text = text.replace("’", "'")
    text = text.replace("‘", " ")
    text = text.replace("\t", " ")
    text = text.replace("!...", " ")
    text = text.replace("....", " ")
    text = text.replace("...", " ")
    text = text.replace(":", " ")
    text = text.replace(";", " ")
    text = text.replace("!", " ")
    text = text.replace(",", " ")
    text = text.replace("?", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("—", " ")
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace(".", " ")
    text = text.replace("&", " and ")
    text = text.replace(" 57E ", " fifty seven e ")
    text = text.replace(" 1st ", " first ")
    text = text.replace(" 6th ", " sixth ")
    text = text.replace(" 1895 ", " eighteen ninety five ")
    text = text.replace(" 1891", " eighteen ninety one")
    text = text.replace("1891", " eighteen ninety one")
    text = text.replace("65 ", " sixty five ")
    if text[-1] == ".":
        text = text[:-1]
    text = re.sub("  +", " ", text)
    return text.lower().strip()

In [13]:
clean = [cleaner(x) for x in sents]

In [14]:
with open("dubliners-clean.txt", "w") as outf:
    for line in clean:
        outf.write(line + "\n")

In [ ]:
!apt install -y build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev

In [16]:
%cd /tmp

/tmp


In [ ]:
!git clone https://github.com/kpu/kenlm

In [18]:
%cd kenlm

/tmp/kenlm


In [ ]:
!mkdir build
%cd build
!cmake ..
!make -j 4

In [20]:
%cd /kaggle/working

/kaggle/working


In [ ]:
!/tmp/kenlm/build/bin/lmplz -o 5 < dubliners-clean.txt > dubliners.arpa